# Function Generation for the Training of λ-Nets

## Package installation (uncommand first line to install packages at the beginning)

In [1]:
%%script false --no-raise-error
#if some errors occur during the installation consider using "sudo pip install"
!pip install numpy
!pip install pandas
!pip install truth-table-generator
!pip install more-itertools
!pip install tqdm
!pip install joblib
!pip install scipy
!pip install PrettyTable
!pip install colored
!pip install scikit-learn
!pip install keras
!pip install ipython
!pip install livelossplot
!pip install matplotlib
!pip install seaborn
!pip install tensorflow
!pip install tensorflow-gpu

## Specitication of Experiment Settings

In [2]:
import math
def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

In [3]:
d = 3  
n = 4
sparsity = nCr(n+d, d)


x_max = 1 
x_min = -1
x_step = 0.01
a_max = 10 
a_min = -10
a_step = 0.1

n_jobs = -3

lambda_dataset_size = 1000 #specify the number of data points to calculate the function values for (determines the lambda net training size)

interpretation_dataset_size = 10000 #specifies the number of functions generated (specifies the interpretation-net dataset size)

same_training_all_lambda_nets = False


In [4]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

RANDOM_SEED = 42

if same_training_all_lambda_nets:
    training_string = '_same'
else:
    training_string = '_diverse'
    
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity)) 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(interpretation_dataset_size))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

Variables: 4 (abcd)
Degree: 3
Sparsity: 35
Lambda-Net Dataset Size: 1000
I-Net Dataset Size: 10000
Coefficient Range: [-10, 10]
Variable Range: [-1, 1]


## Imports

In [5]:
import ttg
from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 
random.seed(42)

import os

directory_names = ['parameters', 'plotting', 'saved_polynomial_lists', 'results', 'saved_models', 'weights', 'weights_training']
if not os.path.exists('./data'):
    os.mkdir('./data')
for directory_name in directory_names:
    path = './data/' + directory_name
    if not os.path.exists(path):
        os.mkdir(path)
        
import seaborn as sns
sns.set_style("darkgrid")
        
import random 
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)




## Utility functions

In [6]:
#test for exact equality
def arreq_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if np.array_equal(elem, myarr)), False)

In [7]:
ALPHABET = \
  "0123456789abcdefghijklmnopqrstuvwxyz"

def encode (n):
    try:
        return ALPHABET [n]
    except IndexError:
        raise Exception ("cannot encode: %s" % n)

def dec_to_base (dec = 0, base = 16):
    if dec < base:
        return encode (dec)
    else:
        return dec_to_base (dec // base, base) + encode (dec % base)

# Function Generation

In [8]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)



List length: 256
Number of monomials in a polynomial with 4 variables and degree 3: 35
Sparsity: 35
['0000', '0001', '0002', '0003', '0010', '0011', '0012', '0013', '0020', '0021', '0022', '0023', '0030', '0031', '0032', '0033', '0100', '0101', '0102', '0103', '0110', '0111', '0112', '0113', '0120', '0121', '0122', '0123', '0130', '0131', '0132', '0133', '0200', '0201', '0202', '0203', '0210', '0211', '0212', '0213', '0220', '0221', '0222', '0223', '0230', '0231', '0232', '0233', '0300', '0301', '0302', '0303', '0310', '0311', '0312', '0313', '0320', '0321', '0322', '0323', '0330', '0331', '0332', '0333', '1000', '1001', '1002', '1003', '1010', '1011', '1012', '1013', '1020', '1021', '1022', '1023', '1030', '1031', '1032', '1033', '1100', '1101', '1102', '1103', '1110', '1111', '1112', '1113', '1120', '1121', '1122', '1123', '1130', '1131', '1132', '1133', '1200', '1201', '1202', '1203', '1210', '1211', '1212', '1213', '1220', '1221', '1222', '1223', '1230', '1231', '1232', '1233', '1


List length: 35
Number of monomials in a polynomial with 4 variables and degree 3: 35
Sparsity: 35
['0000', '0001', '0002', '0003', '0010', '0011', '0012', '0020', '0021', '0030', '0100', '0101', '0102', '0110', '0111', '0120', '0200', '0201', '0210', '0300', '1000', '1001', '1002', '1010', '1011', '1020', '1100', '1101', '1110', '1200', '2000', '2001', '2010', '2100', '3000']


In [9]:
list_of_polynomials = []

if interpretation_dataset_size/((a_max-a_min)*10**int(-np.log10(a_step)))**(nCr(n+d, d)) <= 10e-4:
    while len(list_of_polynomials) < interpretation_dataset_size:
        random_polynomial = list(random_product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
        if random_polynomial not in list_of_polynomials:
            list_of_polynomials.append(random_polynomial)
else:
    all_polynomials_list = list(product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
    list_of_polynomials = [all_polynomials_list[i] for i in np.random.choice(len(all_polynomials_list), interpretation_dataset_size, replace=False)]
    del all_polynomials_list

for polynomial in tqdm(list_of_polynomials):
    sparsity_indices = np.random.choice(nCr(n+d, d), nCr(n+d, d)-sparsity, replace=False)
    for sparsity_index in sparsity_indices:                            
        polynomial[sparsity_index] = 0
                                
polynomials_list_df = pd.DataFrame(data=list_of_polynomials, columns=list_of_monomial_identifiers)

    
print(len(list_of_monomial_identifiers))
print(polynomials_list_df.shape)


35
(10000, 35)


In [10]:
polynomials_list_df.head()

,0000,0001,0002,0003,0010,0011,0012,0020,0021,0030,...,1020,1100,1101,1110,1200,2000,2001,2010,2100,3000
0,6.3,-7.2,-9.4,8.9,-3.0,-3.8,-4.3,-6.5,8.8,-7.4,...,-5.0,8.3,6.6,7.9,3.9,0.7,-4.4,1.4,5.0,-2.9
1,-9.9,9.4,-6.0,7.8,0.8,-1.3,-2.9,-6.1,-4.5,9.5,...,-8.0,4.1,-2.5,6.0,5.8,-0.8,4.7,-5.1,8.0,-8.3
2,-8.9,6.9,-4.2,9.7,-2.6,-8.0,-4.1,-7.5,-0.3,-2.9,...,-5.7,3.6,8.6,-3.8,-5.9,1.8,-0.3,-3.1,6.3,7.6
3,4.2,-4.4,7.5,-1.7,9.6,9.8,-8.6,-4.2,-9.2,-2.0,...,-6.5,-3.7,9.0,4.3,3.7,-3.3,9.1,4.9,0.9,4.9
4,0.2,-0.8,-4.4,-6.5,3.0,2.6,-7.7,9.3,-8.8,-7.2,...,7.4,8.4,-7.1,7.4,3.7,9.2,-3.2,9.6,6.4,-1.3


In [11]:
def generate_random_x_values(size, x_max, x_min, x_step, numnber_of_variables):
    x_values_list = []
    
    for j in range(size):
        values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))
        while arreq_in_list(values, x_values_list):
                values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))         
        x_values_list.append(values)
    
    return x_values_list


In [12]:
if same_training_all_lambda_nets:
    x_values = generate_random_x_values(lambda_dataset_size, x_max, x_min, x_step, n)
    x_values_list = [x_values for _ in range(interpretation_dataset_size)]
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    x_values_list = parallel(delayed(generate_random_x_values)(lambda_dataset_size, x_max, x_min, x_step, n) for _ in range(interpretation_dataset_size))
    del parallel


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed: 18.8min
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed: 22.8min
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed: 27.2min
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed: 31.9min
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed: 37.0min
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed: 42.5min
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [13]:
from functools import reduce

def calcualate_function_with_data(coefficient_list, variable_values):
    
    result = 0    
    for coefficient_value, coefficient_multipliers in zip(coefficient_list, list_of_monomial_identifiers):
        partial_results = [variable_value**int(coefficient_multiplier) for coefficient_multiplier, variable_value in zip(coefficient_multipliers, variable_values)]
        
        result += coefficient_value * reduce(lambda x, y: x*y, partial_results)

    return result, variable_values
 
def calculate_function_values_from_polynomial(true_value_test, evaluation_dataset):
    if isinstance(true_value_test, pd.DataFrame):
        true_value_test = true_value_test.values
        
    true_value_fv = []
    true_value_coeff = []
    
    for evaluation in evaluation_dataset:
        true_function_value, true_coeff = calcualate_function_with_data(true_value_test, evaluation)
       
        true_value_fv.append(true_function_value) 
        true_value_coeff.append(true_coeff)
        
    return [true_value_test, pd.DataFrame(np.array(true_value_coeff))], [true_value_test, pd.DataFrame(np.array(true_value_fv))]


In [14]:
result_list = []

polynomials_X_data_list = []
polynomials_y_data_list = []
    
chunks = max(interpretation_dataset_size//1000, 1)

for polynomials_list_df_chunk in tqdm(np.array_split(polynomials_list_df, chunks), total=chunks):
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_sublist = parallel(delayed(calculate_function_values_from_polynomial)(polynomial, x_values) for (_, polynomial), x_values in zip(polynomials_list_df_chunk.iterrows(), x_values_list))  
    result_list.extend(result_sublist)
    del parallel

polynomials_X_data_list = [result[0] for result in result_list]
polynomials_y_data_list = [result[1] for result in result_list]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   53.2s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   50.2s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   12.8s
[Pa

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   50.4s finished


In [15]:
polynomials_X_data_list[0][0].head()

0000    6.3
0001   -7.2
0002   -9.4
0003    8.9
0010   -3.0
Name: 0, dtype: float64

In [16]:
polynomials_X_data_list[0][1].head()

,0,1,2,3
0,-0.62,-0.95,0.34,-0.76
1,-0.30,0.95,-0.77,0.31
2,-0.30,0.28,0.48,0.08
3,-0.52,0.98,0.23,0.17
4,0.71,-0.92,-0.31,0.26


In [17]:
polynomials_y_data_list[0][0].values

array([ 6.3, -7.2, -9.4,  8.9, -3. , -3.8, -4.3, -6.5,  8.8, -7.4,  7.3,
        8.9,  3.9, -7.8,  5.1,  0.8, -9.2, -9.3, -7.7, -4.5, -4.1,  2.9,
        5.4, -9.4,  4.3, -5. ,  8.3,  6.6,  7.9,  3.9,  0.7, -4.4,  1.4,
        5. , -2.9])

In [18]:
polynomials_y_data_list[0][0].head()

0000    6.3
0001   -7.2
0002   -9.4
0003    8.9
0010   -3.0
Name: 0, dtype: float64

In [19]:
polynomials_y_data_list[0][1].head()

,0
0,7.283630
1,11.515334
2,3.999790
3,-8.337609
4,-17.255761


In [20]:
polynomials_X_data_list[0]

[0000    6.3
 0001   -7.2
 0002   -9.4
 0003    8.9
 0010   -3.0
 0011   -3.8
 0012   -4.3
 0020   -6.5
 0021    8.8
 0030   -7.4
 0100    7.3
 0101    8.9
 0102    3.9
 0110   -7.8
 0111    5.1
 0120    0.8
 0200   -9.2
 0201   -9.3
 0210   -7.7
 0300   -4.5
 1000   -4.1
 1001    2.9
 1002    5.4
 1010   -9.4
 1011    4.3
 1020   -5.0
 1100    8.3
 1101    6.6
 1110    7.9
 1200    3.9
 2000    0.7
 2001   -4.4
 2010    1.4
 2100    5.0
 3000   -2.9
 Name: 0, dtype: float64,         0     1     2     3
 0   -0.62 -0.95  0.34 -0.76
 1   -0.30  0.95 -0.77  0.31
 2   -0.30  0.28  0.48  0.08
 3   -0.52  0.98  0.23  0.17
 4    0.71 -0.92 -0.31  0.26
 5    0.16 -0.87  0.43 -0.49
 6    0.30  0.54  0.61  0.59
 7   -0.59 -0.09 -0.73 -0.31
 8    0.04  0.80  0.50 -0.49
 9    0.11 -0.66 -0.95 -0.89
 10  -0.17 -0.51  0.34  0.48
 11   0.71  0.12 -0.63 -0.40
 12   0.81  0.49  0.72  0.40
 13  -0.02  0.70  0.38  0.24
 14   0.89  0.52 -0.33  0.65
 15  -0.95 -0.15  0.22  0.41
 16  -0.60  0.42  0.61 -0.5

In [21]:
polynomials_y_data_list[0]

[0000    6.3
 0001   -7.2
 0002   -9.4
 0003    8.9
 0010   -3.0
 0011   -3.8
 0012   -4.3
 0020   -6.5
 0021    8.8
 0030   -7.4
 0100    7.3
 0101    8.9
 0102    3.9
 0110   -7.8
 0111    5.1
 0120    0.8
 0200   -9.2
 0201   -9.3
 0210   -7.7
 0300   -4.5
 1000   -4.1
 1001    2.9
 1002    5.4
 1010   -9.4
 1011    4.3
 1020   -5.0
 1100    8.3
 1101    6.6
 1110    7.9
 1200    3.9
 2000    0.7
 2001   -4.4
 2010    1.4
 2100    5.0
 3000   -2.9
 Name: 0, dtype: float64,              0
 0     7.283630
 1    11.515334
 2     3.999790
 3    -8.337609
 4   -17.255761
 5    -2.057307
 6    -4.735710
 7     6.258367
 8    -6.836901
 9    -9.320159
 10   -8.246242
 11    8.236377
 12   -7.605187
 13   -2.028823
 14   12.097666
 15    5.679797
 16    3.971913
 17   -0.161452
 18  -27.481118
 19  -16.648209
 20   -7.098924
 21   12.783471
 22   -2.955187
 23   -1.436107
 24   -7.898406
 25    3.708957
 26   -9.723849
 27    0.117193
 28   11.432142
 29    4.598385
 ..         ...
 970   3

In [22]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample' + str(interpretation_dataset_size) + '_variables_' + str(n) +  '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(polynomials_X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(polynomials_y_data_list, f)#, protocol=2)
